### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [ ]:
# Write your code from here